In [1]:
import os
import numpy as np
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from sqlalchemy.sql import select, column, text
from sqlalchemy.sql.expression import func
from flask import (
    Flask,
    render_template,
    jsonify,
    request,
    redirect)
#from models import create_classes
import simplejson
from flask_sqlalchemy import SQLAlchemy
from login_details import username
from login_details import password

In [2]:
rds_connection_string = f"{username}:{password}@localhost:5432/project_2"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [3]:
names = engine.table_names()

In [4]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [5]:
# Save references to each table
Division = Base.classes.electoral_division
Results = Base.classes.election_results
Vote_Types = Base.classes.election_vote_types
Turnout = Base.classes.election_turnout
Marriage_Results = Base.classes.marriage_postal_results
Marriage_Turnout = Base.classes.marriage_postal_turnout
Marriage_Participants_Age = Base.classes.marriage_postal_participants_by_age
#Age = Base.classes."2017_population_agedemo"
Cultural_Diversity = Base.classes.cultural_diversity
Education = Base.classes.education
Labor_Liberal = Base.classes.labor_liberal_votes

In [6]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [7]:
print(names)

['electoral_division', 'election_results', 'election_vote_types', 'election_turnout', 'marriage_postal_results', 'marriage_postal_turnout', 'marriage_postal_participants_by_age', '2017_population_agedemo', 'cultural_diversity', 'education', 'labor_liberal_votes']


In [8]:
# Map data 
map_results = session.query(Division.division_id, Division.electoral_division, Division.state, Marriage_Results.division_id, Marriage_Results.yes_count, Marriage_Results.no_count, Results.division_id, Results.successful_party, Marriage_Turnout.division_id, Marriage_Turnout.turnout_percent)\
                    .join(Marriage_Results, Marriage_Results.division_id == Division.division_id)\
                    .join(Results, Results.division_id == Division.division_id)\
                    .join(Marriage_Turnout, Marriage_Turnout.division_id == Division.division_id).all()

In [9]:
map_df = pd.DataFrame(map_results)
map_df.head()

,division_id,electoral_division,state,division_id,yes_count,no_count,division_id,successful_party,division_id,turnout_percent
0,179,Adelaide,SA,179,62769,26771,179,Australian Labor Party,179,81.339000
1,197,Aston,VIC,197,48455,29730,197,Liberal,197,81.578533
2,198,Ballarat,VIC,198,65613,27405,198,Australian Labor Party,198,81.665902
3,103,Banks,NSW,103,37736,46343,103,Liberal,103,80.095910
4,180,Barker,SA,180,42498,38827,180,Liberal,180,77.043031


In [10]:

# creating html script using to_html
map_df.to_html('map.html', index = False)

In [11]:
# Bubble data
bubble_results = session.query(Division.division_id, Division.electoral_division, Division.state, Marriage_Results.division_id, Marriage_Results.yes_count, Marriage_Results.total_responses, Labor_Liberal.division_id, Labor_Liberal.liberal_percent, Labor_Liberal.labor_percent, Education.division_id, Education.year_12_completion_percent, Education.higher_education_completion_percent)\
                    .join(Marriage_Results, Marriage_Results.division_id == Division.division_id)\
                    .join(Labor_Liberal, Labor_Liberal.division_id == Division.division_id)\
                    .join(Education, Education.division_id == Division.division_id).all()

In [12]:
bubble_df = pd.DataFrame(bubble_results)
bubble_df.head()

,division_id,electoral_division,state,division_id,yes_count,total_responses,division_id,liberal_percent,labor_percent,division_id,year_12_completion_percent,higher_education_completion_percent
0,179,Adelaide,SA,179,62769,89540,179,45.35,54.65,179,82.4,49.3
1,197,Aston,VIC,197,48455,78185,197,58.59,41.41,197,79.7,49.3
2,198,Ballarat,VIC,198,65613,93018,198,42.68,57.32,198,68.6,45.1
3,103,Banks,NSW,103,37736,84079,103,51.44,48.56,103,83.4,47.8
4,180,Barker,SA,180,42498,81325,180,65.19,34.81,180,57.9,34.9


In [13]:

# creating html script using to_html
bubble_df.to_html('bubble.html', index = False)

In [14]:
# Pie Data
pie_results = session.query(Division.division_id, Division.electoral_division, Division.state, Marriage_Results.division_id, Marriage_Results.yes_count, Marriage_Results.no_count)\
                    .join(Marriage_Results, Marriage_Results.division_id == Division.division_id).all()

In [15]:
pie_df = pd.DataFrame(pie_results)
pie_df.head()

,division_id,electoral_division,state,division_id,yes_count,no_count
0,179,Adelaide,SA,179,62769,26771
1,197,Aston,VIC,197,48455,29730
2,198,Ballarat,VIC,198,65613,27405
3,103,Banks,NSW,103,37736,46343
4,180,Barker,SA,180,42498,38827


In [16]:

# creating html script using to_html
pie_df.to_html('pie.html', index = False)

In [17]:
# Bar Data
bar_results = session.query(Division.division_id, Division.electoral_division, Division.state, Marriage_Participants_Age.division_id, Marriage_Participants_Age.ages_18_34, Marriage_Participants_Age.ages_35_49, Marriage_Participants_Age.ages_50_64, Marriage_Participants_Age.ages_65_79, Marriage_Participants_Age.ages_80_plus)\
                    .join(Marriage_Participants_Age, Marriage_Participants_Age.division_id == Division.division_id).all()


In [18]:
bar_df = pd.DataFrame(bar_results)
bar_df.head()

,division_id,electoral_division,state,division_id,ages_18_34,ages_35_49,ages_50_64,ages_65_79,ages_80_plus
0,179,Adelaide,SA,179,24474,20770,22353,15429,5396
1,197,Aston,VIC,197,20745,18667,21236,13420,3638
2,198,Ballarat,VIC,198,22345,22008,24709,18264,5257
3,103,Banks,NSW,103,20562,20416,22008,14571,6289
4,180,Barker,SA,180,14079,16493,24109,20212,6231


In [19]:
# creating html script using to_html
bar_df.to_html('bar.html', index = False)